- Autor: Virginia Monroy Malca
- Este notebook es una adaptación del contenido de Alain Chabrier
- https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/7d1e0d95-3386-4906-970d-029525bd743c/view?access_token=13618f0954cce214d8a0b1abf192af4bedbc99afd048f810df278d8f4f984253

# OPTIMIZACIÓN DE OPERACIONES

Este tutorial incluye todo lo que necesita para configurar el Modelado CPLEX de IBM Decision Optimization para Python (DOcplex), crear un modelo de Programación matemática y obtener su solución resolviendo el modelo con IBM ILOG CPLEX Optimizer.

Tabla de contenido:

-  [Describiendo el problema del negocio](#Describiendo-el-problema-del-negocio)
*  [Prepación de los datos](#Preparación-de-los-datos)
*  [Use decision optimization](#Use-IBM-Decision-Optimization-CPLEX-Modeling-for-Python)
    -  [Paso 1: Configurar el modelo](#Paso-1:-Configurar-el-modelo)
        * [Define las variables de decisión](#Define-las-variables-de-decisión)
        * [Define las restricciones](#Define-las-restricciones)
        * [Expresa el objetivo](#Expresa-el-objectivo)
        * [Reseuleve el modelo con DO](#Resuelve-el-modelo)
    *  [Paso 2: Analizar la solución](#Paso-2:-Analizar-la-solución)
*  [Exportar datos](#Exportar-datos)

## Describiendo el problema del negocio
* Imagine que tiene la oportunidad de promocionar nuevos productos a sus clientes existentes. Puede decidir qué producto (y solo uno) ofrecer a cada uno de sus clientes.
* Usted sabe el costo de ejecutar estas promociones y Machine Learning lo ayudó a identificar los ingresos esperados de estos clientes de acuerdo con sus características y utilizando un modelo capacitado con datos históricos.
<br>

Hay que respetar un conjunto de restricciones comerciales:

* Como máximo debe haber una campaña por cliente.
* Como máximo deben haber 3 clientes por campaña.

## Preparación de los datos
* Importamos el token para poder cargar y guardar archivos.

In [27]:
# The code was removed by Watson Studio for sharing.

* Cargarmos el archivo subido en DATA.

In [28]:
import pandas as pd

In [39]:
my_file = project.get_file("candidates.csv")
candidates_df = pd.read_csv(my_file)

In [40]:
candidates_df

,customer,campaign,revenue
0,Customer 1,Home,70
1,Customer 2,Home,80
2,Customer 3,Home,90
3,Customer 4,Home,50
4,Customer 5,Home,100
5,Customer 6,Home,110
6,Customer 7,Home,20
7,Customer 8,Home,10
8,Customer 9,Home,0
9,Customer 10,Home,40


* Extrayendo la lista de clientes y campañas

In [41]:
all_customers = candidates_df.customer.unique()
all_campaigns = candidates_df.campaign.unique()

candidates_df.set_index(['customer', 'campaign'],inplace=True)

In [42]:
all_customers

array(['Customer 1', 'Customer 2', 'Customer 3', 'Customer 4',
       'Customer 5', 'Customer 6', 'Customer 7', 'Customer 8',
       'Customer 9', 'Customer 10'], dtype=object)

In [43]:
all_campaigns

array(['Home', 'Auto', 'Travel'], dtype=object)

## Use IBM Decision Optimization CPLEX Modeling for Python

Cree el modelo de optimización para seleccionar la mejor campaña para cada cliente según sus ingresos.

### Paso 1: Configurar el modelo

Configure el modelo prescriptivo utilizando el paquete de modelado de programación matemática (docplex.mp).

#### Crear el modelo

In [44]:
from docplex.mp.model import Model
mdl = Model(name="Campaigns")

#### Define las variables de decisión
- Crear una variable de decisión binaria por posible asignación (cliente - campaña)

In [45]:
assignments = mdl.binary_var_matrix(keys1=all_customers, keys2=all_campaigns, name=lambda ns: "Assignment %s_%s" % (ns[0],ns[1]))

In [46]:
assignments

{('Customer 1',
  'Home'): docplex.mp.Var(type=B,name='Assignment Customer 1_Home'),
 ('Customer 1',
  'Auto'): docplex.mp.Var(type=B,name='Assignment Customer 1_Auto'),
 ('Customer 1',
  'Travel'): docplex.mp.Var(type=B,name='Assignment Customer 1_Travel'),
 ('Customer 2',
  'Home'): docplex.mp.Var(type=B,name='Assignment Customer 2_Home'),
 ('Customer 2',
  'Auto'): docplex.mp.Var(type=B,name='Assignment Customer 2_Auto'),
 ('Customer 2',
  'Travel'): docplex.mp.Var(type=B,name='Assignment Customer 2_Travel'),
 ('Customer 3',
  'Home'): docplex.mp.Var(type=B,name='Assignment Customer 3_Home'),
 ('Customer 3',
  'Auto'): docplex.mp.Var(type=B,name='Assignment Customer 3_Auto'),
 ('Customer 3',
  'Travel'): docplex.mp.Var(type=B,name='Assignment Customer 3_Travel'),
 ('Customer 4',
  'Home'): docplex.mp.Var(type=B,name='Assignment Customer 4_Home'),
 ('Customer 4',
  'Auto'): docplex.mp.Var(type=B,name='Assignment Customer 4_Auto'),
 ('Customer 4',
  'Travel'): docplex.mp.Var(type=B,na

#### Define las restricciones
- Como máximo debe haber una campaña por cliente.
- Como máximo deben haber 3 clientes por campaña.

In [47]:
# Como máximo una campaña por cliente
for customer in all_customers:       
    print(mdl.add_constraint( mdl.sum(assignments[customer, campaign] for campaign in all_campaigns) <= 1))

Assignment Customer 1_Home+Assignment Customer 1_Auto+Assignment Customer 1_Travel <= 1
Assignment Customer 2_Home+Assignment Customer 2_Auto+Assignment Customer 2_Travel <= 1
Assignment Customer 3_Home+Assignment Customer 3_Auto+Assignment Customer 3_Travel <= 1
Assignment Customer 4_Home+Assignment Customer 4_Auto+Assignment Customer 4_Travel <= 1
Assignment Customer 5_Home+Assignment Customer 5_Auto+Assignment Customer 5_Travel <= 1
Assignment Customer 6_Home+Assignment Customer 6_Auto+Assignment Customer 6_Travel <= 1
Assignment Customer 7_Home+Assignment Customer 7_Auto+Assignment Customer 7_Travel <= 1
Assignment Customer 8_Home+Assignment Customer 8_Auto+Assignment Customer 8_Travel <= 1
Assignment Customer 9_Home+Assignment Customer 9_Auto+Assignment Customer 9_Travel <= 1
Assignment Customer 10_Home+Assignment Customer 10_Auto+Assignment Customer 10_Travel <= 1


In [48]:
##assignments[customer, campaign]

In [49]:
##mdl.sum(assignments[customer, campaign] for campaign in all_campaigns)

In [50]:
# Como máximo tres clientes por campaña
for campaign in all_campaigns:       
    print(mdl.add_constraint( mdl.sum(assignments[customer, campaign] for customer in all_customers) <= 3))

Assignment Customer 1_Home+Assignment Customer 2_Home+Assignment Customer 3_Home+Assignment Customer 4_Home+Assignment Customer 5_Home+Assignment Customer 6_Home+Assignment Customer 7_Home+Assignment Customer 8_Home+Assignment Customer 9_Home+Assignment Customer 10_Home <= 3
Assignment Customer 1_Auto+Assignment Customer 2_Auto+Assignment Customer 3_Auto+Assignment Customer 4_Auto+Assignment Customer 5_Auto+Assignment Customer 6_Auto+Assignment Customer 7_Auto+Assignment Customer 8_Auto+Assignment Customer 9_Auto+Assignment Customer 10_Auto <= 3
Assignment Customer 1_Travel+Assignment Customer 2_Travel+Assignment Customer 3_Travel+Assignment Customer 4_Travel+Assignment Customer 5_Travel+Assignment Customer 6_Travel+Assignment Customer 7_Travel+Assignment Customer 8_Travel+Assignment Customer 9_Travel+Assignment Customer 10_Travel <= 3


In [51]:
##mdl.sum(assignments[customer, campaign] for customer in all_customers)

In [52]:
mdl.print_information()

Model: Campaigns
 - number of variables: 30
   - binary=30, integer=0, continuous=0
 - number of constraints: 13
   - linear=13
 - parameters: defaults
 - problem type is: MILP


#### Expresa el objetivo

Maximice los ingresos esperados.

In [55]:
expected_revenue = mdl.sum(assignments[customer, campaign] * candidates_df.revenue[customer, campaign] for customer in all_customers for campaign in all_campaigns)

mdl.add_kpi(expected_revenue, "Expected Revenue");
mdl.maximize(expected_revenue);


In [53]:
#candidates_df

In [54]:
#candidates_df.revenue[customer, campaign]

In [56]:
#print(mdl.sum(assignments[customer, campaign] * candidates_df.revenue[customer, campaign] for customer in all_customers for campaign in all_campaigns))

#### Resuelve el modelo

In [57]:
s = mdl.solve(log_output=True)
assert s, "solve failed"
mdl.report()

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 1 columns.
Reduced MIP has 13 rows, 29 columns, and 58 nonzeros.
Reduced MIP has 29 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 13 rows, 29 columns, and 58 nonzeros.
Reduced MIP has 29 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 10.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap


* model Campaigns solved with objective = 770
*  KPI: Expected Revenue = 770.000


### Paso 2: Analizar la solución

Primero, muestre la **Campaña óptima por cliente**.

In [69]:
# Extraer el valor de la solución en un dataframe
assignments_df = pd.DataFrame({'assignment': assignments})
assignments_df.index.names=['all_customers', 'all_campaigns']
assignments_df = assignments_df.assignment.apply(lambda v: v.solution_value)

# Mantener solo los assigments seleccionados
assignments_df = assignments_df[assignments_df ==1]
display(assignments_df)

all_customers  all_campaigns
Customer 1     Home             1.0
Customer 10    Travel           1.0
Customer 2     Auto             1.0
Customer 3     Auto             1.0
Customer 4     Home             1.0
Customer 5     Home             1.0
Customer 6     Auto             1.0
Customer 7     Travel           1.0
Customer 8     Travel           1.0
Name: assignment, dtype: float64

# Exportar datos

In [72]:
project.save_data(data=assignments_df.to_csv(index=False),file_name='cliente_campana_opt.csv',overwrite=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


{'file_name': 'df_export.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'workshoppacfico-donotdelete-pr-hwikylsfyocyxe',
 'asset_id': '8ca51730-17e7-454a-9240-44ee5cfba7c8'}